# Extract lifespan information from Amazon reviews

First let's install some useful packages

In [3]:
import pandas as pd
import numpy as np
import nltk
import json
import gzip

Read in reviews for subset of electronics from Amazon

In [4]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    i_max = 1000000
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
        if i > i_max:
            break
        if i % 50000 == 0:
            print(str(int(100*i/i_max)) + '%')
            
    return pd.DataFrame.from_dict(df, orient='index')

reviews = getDF('data/reviews_Electronics_5.json.gz')
reviews.head(2)

5%
10%
15%
20%
25%
30%
35%
40%
45%
50%
55%
60%
65%
70%
75%
80%
85%
90%
95%
100%


,reviewTime,asin,reviewerName,reviewerID,unixReviewTime,helpful,summary,reviewText,overall
0,"06 2, 2013",0528881469,amazdnu,AO94DHGC771SJ,1370131200,"[0, 0]",Gotta have GPS!,We got this GPS for my husband who is an (OTR)...,5.0
1,"11 25, 2010",0528881469,Amazon Customer,AMO214LNFCEI4,1290643200,"[12, 15]",Very Disappointed,"I'm a professional OTR truck driver, and I bou...",1.0


Extract reviews with information on items breaking down
For now: only looking for reviews of the form "worked for X period"

Allowed time periods:
* Day
* Week
* Month
* Year

In [7]:
term = 'worked for'
timeperiod_terms = set(['day','week','month','year'])
break_reviews = reviews[reviews['reviewText'].str.contains(term)]

asins = []
reviewerIDs = []
n_periods = []
t_periods = []
ratings = []
words = []

for index, row in break_reviews.iterrows():
    review = row['reviewText']
    asin = row['asin']
    reviewerID = row['reviewerID']
    rating = row['overall']
    
    i_term = review.find(term)
    words_after = review[i_term:].split()[0:10]
    
    for timeperiod in timeperiod_terms:
        if (timeperiod in words_after) or (timeperiod+'s' in words_after):
            i_term = np.argmax(pd.Series(words_after).str.contains(timeperiod))
            words_between = words_after[2:i_term]
            if len(words_between) > 0:
                try:
                    n_periods_float = float(words_between[-1])
                    asins.append(asin)
                    reviewerIDs.append(reviewerID)
                    n_periods.append(n_periods_float)
                    t_periods.append(timeperiod)
                    ratings.append(rating)
                    words.append(' '.join(words_after))
                except ValueError:
                    if words_between[-1] == 'a':
                        n_periods_float = float(1)
                        asins.append(asin)
                        reviewerIDs.append(reviewerID)
                        n_periods.append(n_periods_float)
                        t_periods.append(timeperiod)
                        ratings.append(rating)
                        words.append(' '.join(words_after))
            
breakdowns = pd.DataFrame({'asin': asins, 'reviewerID': reviewerIDs,
                          'n_period': n_periods, 't_period': t_periods,
                          'rating': ratings, 'words': words})

lifespans = []
for i in range(breakdowns.shape[0]):
    if (breakdowns['t_period'][i] == 'day'):
        lifespans.append(breakdowns['n_period'][i] / 30)
    elif (breakdowns['t_period'][i] == 'week'):
        lifespans.append(breakdowns['n_period'][i] * 7 / 30)
    elif (breakdowns['t_period'][i] == 'month'):
        lifespans.append(breakdowns['n_period'][i])
    elif (breakdowns['t_period'][i] == 'year'):
        lifespans.append(breakdowns['n_period'][i] *365 / 30)
    else:
        print('ajjajaj')

breakdowns['lifespan'] = lifespans
print(breakdowns.head(5))

         asin  n_period  rating      reviewerID t_period  \
0  9888002198       1.0     1.0   ATXRV6Z3N08ZX      day   
1  9888002198       6.0     5.0   AA6BJGZG6XI05     week   
2  B00000J0D5       2.0     1.0   A83BIO3AO15LQ    month   
3  B00003006E       1.0     5.0  A2WYNTUJ91DLZW     year   
4  B00004Z5LR       8.0     5.0  A2BFZD9WRHVO0J    month   

                                               words   lifespan  
0  worked for about 1 day or less, when they stopped   0.033333  
1       worked for me over 6 weeks in Europe and UK.   1.400000  
2  worked for maybe 2 months then died. Tried to ...   2.000000  
3          worked for about a year with no problems.  12.166667  
4       worked for them as a temp for 8 months about   8.000000  


Create dataframe of unique items

In [10]:
uniq_asins = breakdowns['asin'].unique()
breakdown_items = pd.DataFrame({'asin': uniq_asins})
breakdown_items['brand'] = 'null'
breakdown_items['model'] = 'null'
print(breakdown_items.head(3))

         asin brand model
0  9888002198  null  null
1  B00000J0D5  null  null
2  B00003006E  null  null


Write data in json format

In [11]:
def write_to_json(items, reviews, filename):
    with open(filename, 'w') as jsonfile:
        jsonfile.write('{\n')
        
        for index, row in items.iterrows():
            thisitem_reviews = reviews.where(reviews.asin == row.asin).dropna().reset_index(drop=True)
            
            jsonfile.write('\t"' + row['asin'] + '" : {\n')
            jsonfile.write('\t\t"brand" : ' + row['brand'] + ',\n')
            jsonfile.write('\t\t"model" : ' + row['model'] + ',\n')
            jsonfile.write('\t\t"reviews" : [ \n')
            
            for iindex, irow in thisitem_reviews.iterrows():
                if iindex == (thisitem_reviews.shape[0] - 1):
                    jsonfile.write('\t\t{\t"lifespan": ' + str(irow['lifespan']) + ',\n')
                    jsonfile.write('\t\t\t"rating": ' + str(irow['rating']) + '\n')
                    jsonfile.write('\t\t}\n')
                else:
                    jsonfile.write('\t\t{\t"lifespan": ' + str(irow['lifespan']) + ',\n')
                    jsonfile.write('\t\t\t"rating": ' + str(irow['rating']) + '\n')
                    jsonfile.write('\t\t},\n')
                
            jsonfile.write('\t\t]\n')
            jsonfile.write('\t},\n')
        jsonfile.write('}\n')

filename = 'data/amazon.json'
write_to_json(breakdown_items, breakdowns, filename)

Convert mock Amazon data from Young Mi

In [246]:
original_columns = ['ASIN','Rating','Months','Brand_name']
new_columns = ['asin','rating','lifespan','brand']

youngmi_amazon = pd.read_csv('data/ratings_Amazon_same_brandname.csv')[original_columns]
youngmi_amazon.columns = new_columns
youngmi_amazon.head(3)

,asin,rating,lifespan,brand
0,0132793040,5,6,FRUYZ9646H
1,0321732944,5,28,XXSKO4415Q
2,0439886341,1,20,VKNEQ7868Y
3,0439886341,3,1,VKNEQ7868Y
4,0439886341,1,19,VKNEQ7868Y


In [247]:
youngmi_amazon_products = youngmi_amazon[['asin','brand']].drop_duplicates()
youngmi_amazon_products['model'] = 'null'
youngmi_amazon_products.columns = ['asin','brand','model']

youngmi_amazon_products.head(3)

,asin,brand,model
0,0132793040,FRUYZ9646H,null
1,0321732944,XXSKO4415Q,null
2,0439886341,VKNEQ7868Y,null
5,0511189877,XIWNE4324C,null
11,0528881469,VXVKC6828Z,null


In [248]:
def write_to_json_ym(items, reviews, filename):
    with open(filename, 'w') as jsonfile:
        jsonfile.write('{\n')
        
        for index, row in items.iterrows():
            thisitem_reviews = reviews.where(reviews.asin == row.asin).dropna().reset_index(drop=True)
            
            jsonfile.write('\t"' + row['asin'] + '" : {\n')
            jsonfile.write('\t\t"brand" : "' + row['brand'] + '",\n')
            jsonfile.write('\t\t"model" : ' + row['model'] + ',\n')
            jsonfile.write('\t\t"reviews" : [ \n')
            
            for iindex, irow in thisitem_reviews.iterrows():
                if iindex == (thisitem_reviews.shape[0] - 1):
                    jsonfile.write('\t\t{\t"lifespan": ' + str(irow['lifespan']) + ',\n')
                    jsonfile.write('\t\t\t"rating": ' + str(irow['rating']) + '\n')
                    jsonfile.write('\t\t}\n')
                else:
                    jsonfile.write('\t\t{\t"lifespan": ' + str(irow['lifespan']) + ',\n')
                    jsonfile.write('\t\t\t"rating": ' + str(irow['rating']) + '\n')
                    jsonfile.write('\t\t},\n')
                
            jsonfile.write('\t\t]\n')
            jsonfile.write('\t},\n')
        jsonfile.write('}\n')

youngmiamazon_filename = 'data/youngmi_amazon.json'
write_to_json_ym(youngmi_amazon_products, youngmi_amazon, youngmiamazon_filename)

Convert mock lab equipment data from Young Mi

In [241]:
original_columns = ['vwr.catalogue','months','ratings']
new_columns = ['asin','lifespan','rating']

youngmi_lab = pd.read_csv('data/vwr.data.csv')
youngmi_lab.columns = new_columns
youngmi_lab.head(3)

,asin,lifespan,rating
0,129-8922,75,1
1,472-5350,29,2
2,527-6106,24,5
3,537-7921,14,4
4,665-1608,40,5


In [242]:
youngmi_lab_products = youngmi_lab[['asin']].drop_duplicates()
youngmi_lab_products['brand'] = 'null'
youngmi_lab_products['model'] = 'null'
youngmi_lab_products.columns = ['asin','brand','model']

youngmi_lab_products.head(3)

,asin,brand,model
0,129-8922,null,null
1,472-5350,null,null
2,527-6106,null,null
3,537-7921,null,null
4,665-1608,null,null


In [243]:
youngmi_lab_filename = 'data/youngmi_lab.json'
write_to_json(youngmi_lab_products, youngmi_lab, youngmi_lab_filename)